In [1]:
import mand.core

from mand.core import Entity, node, Context

from mand.core import ObjectDb, DynamoDbDriver, ddb, _tr, Timestamp, Context
from mand.core import RefData, RefDataUpdateEvent, Workbook, PrintMonitor, SummaryMonitor

rawdb = DynamoDbDriver(ddb)
db = ObjectDb(rawdb)

pClock = _tr.Clock('Portfolio', db=db).write()

In [2]:
class Book(Workbook):
    
    def books(self):
        return [self]

    def prn(self, depth=0):
        print '  ' * depth, self.meta.name()
            
class PortfolioUpdateEvent(RefDataUpdateEvent):

    @node(stored=True)
    def children(self):
        return []
    
class Portfolio(RefData):

    evCls = PortfolioUpdateEvent
    
    @node
    def clock(self):
        return _tr.Clock.get('Portfolio', db=self.meta.db)
    
    @node
    def children(self):
        evs = self.activeEvents()
        if evs:
            return evs[-1].children()
        else:
            return []
        
    def setChildren(self, children, validTime=None, amends=[]):
        ev = self.evCls(entity=self, amends=amends, children=children, db=self.meta.db)
        ev.write(validTime=validTime)
        return ev
    
    @node
    def books(self):
        books = set()
        for c in self.children():
            for b in c.books():
                if b in books:
                    print 'Oops: book appears multiple times'
                books.add(b)
        return list(books)

    def prn(self, depth=0):
        print '  ' * depth, self.meta.name()
        for c in self.children():
            c.prn(depth+1)
            
_tr.add(Book)
_tr.add(Portfolio)
_tr.add(PortfolioUpdateEvent)

In [3]:
with db:
    checking = Book('checking').write()
    savings = Book('savings').write()
    brokerage = Book('brokerage').write()
    misc = Book('misc').write()
    margin = Book('margin').write()
    retirement = Book('401K').write()
    kid1Trust = Book('trust1').write()
    kid2Trust = Book('trust2').write()

    pAll = Portfolio('Family').write()
    pKids = Portfolio('Kids').write()
    pBanking = Portfolio('Banking').write()
    pTrading = Portfolio('Trading').write()
    pDerivs = Portfolio('Derivs').write()

def info():
    c = [ o.meta.name() for o in pAll.children() ]
    b = [ o.meta.name() for o in pAll.books() ]
    print 'All children:', c
    print 'All books   :', b

In [4]:
info()

All children: []
All books   : []


In [5]:
ctx2 = Context({})
with ctx2:
    info()

All children: []
All books   : []


In [6]:
pKids.setChildren([kid1Trust, kid2Trust])
pBanking.setChildren([checking, savings])
pTrading.setChildren([brokerage, margin, retirement])
pDerivs.setChildren([pTrading])
pAll.setChildren([pBanking, pDerivs, pKids, misc])

In [7]:
n = Context.current().getCBM(pAll.books)
info()
n.printInputGraph()
ts = Timestamp()

All children: ['Banking', 'Derivs', 'Kids', 'misc']
All books   : ['savings', '401K', 'brokerage', 'checking', 'trust2', 'trust1', 'margin', 'misc']
 <Portfolio@112146bd0/Portfolio:books in Root>
   <Portfolio@112119410/Portfolio:books in Root>
     <Portfolio@112119410/Portfolio:children in Root>
       <Portfolio@112119410/Portfolio:clock in Root>
       <Clock@112071450/Clock:cutoffs in Root>
         <Clock@112071450/Clock:parent in Root>
           <Clock@112071450/Entity:clock in Root>
           <RootClock@112071210/RootClock:cutoffs in Root>
             <CosmicAll@11218f3d0/CosmicAll:dbState in Root>
             <RootClock@112071210/RootClock:cosmicAll in Root>
         <RootClock@112071210/RootClock:cutoffs in Root>
           <CosmicAll@11218f3d0/CosmicAll:dbState in Root>
           <RootClock@112071210/RootClock:cosmicAll in Root>
       <PortfolioUpdateEvent@112146310/PortfolioUpdateEvent:children in Root>
       <PortfolioUpdateEvent@112146310/Event:amends in Root>
   <

In [8]:
with Context({pClock.cutoffs: ts}):
    b = pAll.books()
    print
    print
    n = Context.current().getCBM(pAll.books)
    n.printInputGraph()
    info()



 <Portfolio@112146bd0/Portfolio:books in Root:4520525720>
   <Portfolio@112146c10/Portfolio:books in Root:4520525720>
     <Portfolio@112146c10/Portfolio:children in Root:4520525720>
       <Portfolio@112146c10/Portfolio:clock in Root:4520525720>
       <PortfolioUpdateEvent@112146350/Event:amends in Root:4520525720>
       <PortfolioUpdateEvent@112146350/PortfolioUpdateEvent:children in Root:4520525720>
       <Clock@112071450/Clock:cutoffs in Root:4520525720>
   <Portfolio@112119410/Portfolio:books in Root:4520525720>
     <Portfolio@112119410/Portfolio:children in Root:4520525720>
       <PortfolioUpdateEvent@112146310/PortfolioUpdateEvent:children in Root:4520525720>
       <Clock@112071450/Clock:cutoffs in Root:4520525720>
       <Portfolio@112119410/Portfolio:clock in Root:4520525720>
       <PortfolioUpdateEvent@112146310/Event:amends in Root:4520525720>
   <Portfolio@112146bd0/Portfolio:children in Root:4520525720>
     <PortfolioUpdateEvent@1120d24d0/PortfolioUpdateEvent:chi

In [9]:
# ctx2 has open time cutoffs, so it should depend on the dbState, thus this should recompute...
with SummaryMonitor():
    with ctx2:
        info()

All children: ['Banking', 'Derivs', 'Kids', 'misc']
All books   : ['savings', '401K', 'brokerage', 'checking', 'trust2', 'trust1', 'margin', 'misc']
Compute activity:
              GetValue:    49
         GetValue/Calc:    17


In [10]:
# and now be cached...
with PrintMonitor():
    with ctx2:
        info()

 Context enter ctx: Root:4520525720
   GetValue begin ctx: Root:4520525720, key: Portfolio@112146bd0/Portfolio:children
     GetValue from ctx value: [<__main__.Portfo..., ctx: Root:4520525720, key: Portfolio@112146bd0/Portfolio:children
   GetValue begin ctx: Root:4520525720, key: Portfolio@112146bd0/Portfolio:books
     GetValue from ctx value: [<__main__.Book o..., ctx: Root:4520525720, key: Portfolio@112146bd0/Portfolio:books
All children: ['Banking', 'Derivs', 'Kids', 'misc']
All books   : ['savings', '401K', 'brokerage', 'checking', 'trust2', 'trust1', 'margin', 'misc']


In [11]:
pAll.prn()

 Family
   Banking
     checking
     savings
   Derivs
     Trading
       brokerage
       margin
       401K
   Kids
     trust1
     trust2
   misc


In [12]:
pDerivs.setChildren([pTrading, misc])     
info()
pAll.prn()

Oops: book appears multiple times
All children: ['Banking', 'Derivs', 'Kids', 'misc']
All books   : ['savings', '401K', 'brokerage', 'misc', 'trust2', 'trust1', 'margin', 'checking']
 Family
   Banking
     checking
     savings
   Derivs
     Trading
       brokerage
       margin
       401K
     misc
   Kids
     trust1
     trust2
   misc
